In [17]:
#import required packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
#read the data
df = pd.read_csv("AirQualityUCI.csv", parse_dates=[['Date', 'Time']], delimiter=';')

C:\Users\Digitweb Sathurjan\AppData\Local\Temp\ipykernel_22844\430461545.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("AirQualityUCI.csv", parse_dates=[['Date', 'Time']], delimiter=';')


In [19]:
#check the dtypes
print(df.dtypes)

Date_Time         object
CO(GT)            object
PT08.S1(CO)        int64
NMHC(GT)           int64
C6H6(GT)          object
PT08.S2(NMHC)      int64
NOx(GT)            int64
PT08.S3(NOx)       int64
NO2(GT)            int64
PT08.S4(NO2)       int64
PT08.S5(O3)        int64
T                 object
RH                object
AH                object
Unnamed: 15      float64
Unnamed: 16      float64
dtype: object


In [20]:
df['Date_Time'] = pd.to_datetime(df.Date_Time , format = '%d/%m/%Y %H.%M.%S')
data = df.drop(['Date_Time'], axis=1)
data.index = df.Date_Time

In [21]:
data.drop(['Unnamed: 15','Unnamed: 16'],axis=1, inplace=True)

In [22]:
#missing value treatment
cols = data.columns
for j in cols:
    for i in range(0,len(data)):
       if data[j][i] == -200:
           data[j][i] = data[j][i-1]

C:\Users\Digitweb Sathurjan\AppData\Local\Temp\ipykernel_22844\3078187597.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data[j][i] == -200:
C:\Users\Digitweb Sathurjan\AppData\Local\Temp\ipykernel_22844\3078187597.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[j][i] = data[j][i-1]
C:\Users\Digitweb Sathurjan\AppData\Local\Temp\ipykernel_22844\3078187597.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[j][i]

In [27]:
#creating the train and validation set
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

In [47]:
train['CO(GT)']=pd.to_numeric(train['CO(GT)'])
train['C6H6(GT)']=pd.to_numeric(train['C6H6(GT)'])
train['T']=pd.to_numeric(train['T'])
train['RH']=pd.to_numeric(train['RH'])
train['AH']=pd.to_numeric(train['AH'])

ValueError: Unable to parse string "11,9" at position 0

In [28]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [ ]:
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', sqrt(mean_squared_error(pred[i], valid[i])))

In [ ]:
#make final predictions
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)